In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:

import numpy as np 
import pandas as pd 
import torch
import math
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt

import os
import seaborn as sn
import json

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential, Model 
from tensorflow.keras import layers
from tensorflow.keras.layers import Conv2D, GlobalAveragePooling2D, Dense, Flatten, Dropout, BatchNormalization
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau

%matplotlib inline


In [ ]:
#tf.keras.models.load_model("../input/casavaleafdiseasemodel-tf/CasavaLeafDiseaseModel_epoch_12_acc_85.h5")

In [ ]:
import math, re, os
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau

import tensorflow as tf
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from kaggle_datasets import KaggleDatasets
from tensorflow import keras
from functools import partial
from sklearn.model_selection import train_test_split

from sklearn.utils import shuffle
import cv2
from tensorflow.keras import applications
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Dense, Dropout, BatchNormalization, GlobalAveragePooling2D
train_csv_path = "../input/cassava-leaf-disease-classification/train.csv"

In [ ]:
train_csv_path = "../input/cassava-leaf-disease-classification/train.csv"
label_json_path = "../input/cassava-leaf-disease-classification/label_num_to_disease_map.json"
images_dir_path = "../input/cassava-leaf-disease-classification/train_images"

In [ ]:
train_csv = pd.read_csv(train_csv_path)

train_csv['label'] = train_csv['label'].astype('string')

label_class = pd.read_json(label_json_path, orient = 'index')

label_class = label_class.values.flatten().tolist()

In [ ]:
train_csv

In [ ]:
train_data_label_3 = train_csv[train_csv["label"]=='3']
train_data_label_2 = train_csv[train_csv["label"]=='2']
train_data_label_1 = train_csv[train_csv["label"]=='1']
train_data_label_0 = train_csv[train_csv["label"]=='0']
train_data_label_4 = train_csv[train_csv["label"]=='4']

In [ ]:
print(len(train_data_label_3))
print(len(train_data_label_2)) 
print(train_data_label_1) 
print(train_data_label_0 )
print(train_data_label_4)

In [ ]:
BATCH_SIZE = 24
IMG_SIZE = 300

In [ ]:
train_data_label_0 = train_csv[train_csv["label"]=='0']
train_data_label_0 = shuffle(train_data_label_0)
train_data_label_1 = train_csv[train_csv["label"]=='1']
train_data_label_1 = shuffle(train_data_label_1)
train_csv1 = pd.concat([train_data_label_1, train_data_label_0], ignore_index=True)

In [ ]:
#train_data_label_3 = train_csv[train_csv["label"]=='3']
#train_data_label_3 = shuffle(train_data_label_3)
#train_data_label_3= train_data_label_3[:4000]
#train_data_label_not_3 = train_csv[train_csv["label"]!='3']
#train_csv = pd.concat([train_data_label_3, train_data_label_not_3], ignore_index=True)

In [ ]:
train_csv

In [ ]:
train_csv = train_csv.append(train_csv1, ignore_index = True)

In [ ]:
train_csv

# ****Data Agumentation and Pre-Processing****

In [ ]:
train_ex = ImageDataGenerator(rotation_range=270,
                                width_shift_range=0.2,
                                height_shift_range=0.2,
                                brightness_range=[0.1,0.9],shear_range=25,
                                zoom_range=0.3,
                                channel_shift_range=0.1,
                                horizontal_flip=True,
                                zca_whitening=True,
                                vertical_flip=True,
                                rescale=1/255,
                                validation_split=0.2
                               )
valid_ex = ImageDataGenerator(rescale=1/255,
                               validation_split = 0.2
                              )

In [ ]:
train_generator = train_ex.flow_from_dataframe(
                            dataframe=train_csv,
                            directory = images_dir_path,
                            x_col = "image_id",
                            y_col = "label",
                            target_size = (IMG_SIZE, IMG_SIZE),
                            class_mode = "categorical",
                            batch_size = BATCH_SIZE,
                            shuffle = True,
                            subset = "training"

)

valid_generator = valid_ex.flow_from_dataframe(
                            dataframe=train_csv,
                            directory = images_dir_path,
                            x_col = "image_id",
                            y_col = "label",
                            target_size = (IMG_SIZE, IMG_SIZE),
                            class_mode = "categorical",
                            batch_size = BATCH_SIZE,
                            shuffle = False,
                            subset = "validation"
)

batch = next(train_generator)
images = batch[0]
labels = batch[1]

plt.figure(figsize=(15,9))
for i, (img, label) in enumerate(zip(images, labels)):
    plt.subplot(5,3, i%15 +1)
    plt.axis('off')
    plt.imshow(img)
    plt.title(label_class[np.argmax(label)])
    
    if i==15:
        break

In [ ]:
def customModel(weights = None):
    
    base_model = keras.applications.InceptionResNetV2(include_top = False,weights = weights, input_shape = [IMG_SIZE,IMG_SIZE,3])
    model = Sequential()
    model.add(base_model)
    model.add(BatchNormalization(axis=-1))
    model.add(GlobalAveragePooling2D())
    model.add(Dropout(0.5))
    model.add(Dense(256,activation = "relu"))
    model.add(Dense(5, activation = "softmax"))
    
    model.compile(loss = "categorical_crossentropy", optimizer = Adam(lr = 0.001), metrics = ["acc"])
    return model

In [ ]:
model = customModel(weights = "imagenet")
#model.summary()

In [ ]:
#LEARNING RATE SCHEDULER
def lr_scheduler(epoch,lr):
    if epoch > 3:
        return lr/1.125
    return lr

early_stopping = EarlyStopping(monitor = "acc", mode = "max",patience = 7)
lr_scheduler = tf.keras.callbacks.LearningRateScheduler(lr_scheduler)
check_point = tf.keras.callbacks.ModelCheckpoint("./CasavaLeafDiseaseDetectModel.h5", 
                                               monitor='val_loss',save_best_only=True)
history = model.fit(train_generator,
                    validation_data = valid_generator,
                    epochs = 25,
                    callbacks = [early_stopping, lr_scheduler, check_point]
                   )


In [ ]:
acc = history.history["acc"]
v_acc = history.history["val_acc"]

loss = history.history["loss"]
v_loss = history.history["loss"]

epochs = range(25)

plt.figure(figsize = (8, 8))
plt.subplot(1, 2, 1)
plt.plot(epochs, acc, label = "Training Set Accuracy")
plt.plot(epochs, v_acc, label = "Validation Set Accuracy")
plt.legend(loc = "lower right")
plt.title("Training and Validation Accuracy")

plt.subplot(1, 2, 2)
plt.plot(epochs, loss, label = "Training Loss")
plt.plot(epochs, v_loss, label = "Validation Loss")
plt.legend(loc = "lower right")
plt.title("Training and Validation Loss")
plt.show()

In [ ]:
model = model = tf.keras.models.load_model("../input/casavaleafdiseasemodel-tf/CasavaLeafDiseaseModel_epoch_12_acc_85.h5")

In [ ]:
preds = []
ss = pd.read_csv('../input/cassava-leaf-disease-classification/sample_submission.csv')

for image in ss.image_id:
    img = tf.keras.preprocessing.image.load_img('../input/cassava-leaf-disease-classification/test_images/' + image)
    img = tf.keras.preprocessing.image.img_to_array(img)
    img = tf.keras.preprocessing.image.smart_resize(img, (IMG_SIZE, IMG_SIZE))
    img = tf.reshape(img, (-1, IMG_SIZE, IMG_SIZE, 3))
    prediction = model.predict(img/255)
    preds.append(np.argmax(prediction))

my_submission = pd.DataFrame({'image_id': ss.image_id, 'label': preds})
my_submission.to_csv('submission.csv', index=False) 

def lr_scheduler(epoch,lr):
    if epoch > 3:
        return lr/1.125
    return lr

early_stopping = EarlyStopping(monitor = "acc", mode = "max",patience = 7)
lr_scheduler = tf.keras.callbacks.LearningRateScheduler(lr_scheduler)
check_point = tf.keras.callbacks.ModelCheckpoint("./CasavaLeafDiseaseModel.h5",  
                                               monitor='val_loss',save_best_only=True)

model  = tf.keras.models.load_model("../input/cassava-leaf-disease-models/EfNetB0_300_16.h5")


model.compile(loss=tf.keras.losses.CategoricalCrossentropy(), optimizer=tf.keras.optimizers.SGD(learning_rate=0.01, momentum=0.9), metrics=['acc', tf.keras.metrics.TruePositives(name='tp')])
model.summary()

try:
    model = tf.keras.models.load_model("../input/casavaleafdiseasemodel-tf/CasavaLeafDiseaseModel_epoch_12_acc_85.h5")
except:
    print("No saved model. So Train the model !")
    
history = model.fit(train_generator, validation_data=valid_generator, epochs=8, callbacks=[check_point, lr_scheduler])

test_img_path = "../input/cassava-leaf-disease-classification/test_images/2216849948.jpg"

img = cv2.imread(test_img_path)
resized_img = cv2.resize(img, (IMG_SIZE, IMG_SIZE)).reshape(-1, IMG_SIZE, IMG_SIZE, 3)/255



p = model.predict()

model = tf.keras.Sequential()
model.add(base)
model.add(BatchNormalization(axis=-1))
model.add(GlobalAveragePooling2D())
model.add(Dropout(0.5))
model.add(Dense(256, activation='relu'))
model.add(Dense(5, activation='softmax'))

model.compile(loss=tf.keras.losses.CategoricalCrossentropy(), optimizer=tf.keras.optimizers.SGD(learning_rate=0.01, momentum=0.9), metrics=['acc', tf.keras.metrics.TruePositives(name='tp')])

# Loading the ResNet152V2 architecture with imagenet weights as base
base = tf.keras.applications.ResNet152V2(include_top=False, weights='imagenet',input_shape=[IMG_SIZE,IMG_SIZE,3])